# Manufacturer / Aircraft Type Heatmap

This notebook produces a heatmap of manufacturers by aicraft models.

Individual models within a family are grouped:

- e.g. Both A320-214 and A320-232 will appear under the "A320" family
- Aircraft with fewer sightings than the configured threshold are assigned to the "Other" family

In [5]:
%run ../pathutils.ipynb
%run ../definitions.ipynb
%run ../database.ipynb
%run ../export.ipynb
%run utils.ipynb

In [6]:
# Sightings threshold below which aircraft are assigned to a generic "Other" group
prefix_count_threshold = 100

In [7]:
import matplotlib.pyplot as plt
import seaborn as sns

def export_heatmap(export_folder_path, data):
    # Plot the heatmap
    fig = plt.figure(figsize=(12, data.shape[0] / 3))
    sns.heatmap(data, annot=False, fmt="d", cmap="YlOrRd")
    plt.title("Aircraft Sightings by Manufacturer and Group (Auto-Generated)")
    plt.ylabel("Manufacturer")
    plt.xlabel("Aircraft Group")
    plt.xticks(rotation=90)
    plt.tight_layout()

    # Export the chart
    export_chart(export_folder_path, "manufacturer-aircraft-heatmap", "png")

    # Close the figure
    plt.close(fig)

In [8]:
# Construct the query
query = construct_query("aircraft", "sightings.sql", {})

try:
    # Run the query to load the data - this raises a ValueError if there's no data
    df = query_data("aircraft", query)

    # Extract an aircraft type prefix for each sighting
    df["Type"] = df["Model"].apply(extract_prefix)

    # Identify counts by prefix and use them to identify common prefixes
    prefix_counts = df["Type"].value_counts()
    common_prefixes = prefix_counts[prefix_counts >= prefix_count_threshold].index

    # Replace uncommon prefixes with "Other"
    df["Type"] = df.apply(
        lambda row: row["Type"] if row["Type"] in common_prefixes else "Other",
        axis=1
    )

    # Generate the pivot table to act as the source data
    pivot_table = df.pivot_table(
        index="Manufacturer",
        columns="Type",
        values="Id",
        aggfunc="count",
        fill_value=0
    )

    # Export the data to excel
    export_folder_path = get_export_folder_path()
    export_to_spreadsheet(export_folder_path, "manufacturer-aircraft-heatmap.xlsx", { "Pivot Table": pivot_table })

    # Export the heatmap
    export_heatmap(export_folder_path, pivot_table)

except ValueError:
    print(f"WARNING: No data found")